In [ ]:
!pip install tensorflow-gpu==1.15.0
!pip install keras==2.0.8
!pip install h5py==2.10.0

     |████████████████████████████████| 411.5 MB 8.4 kB/s 
     |████████████████████████████████| 3.8 MB 41.3 MB/s 
     |████████████████████████████████| 503 kB 41.3 MB/s 
     |████████████████████████████████| 50 kB 7.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=8c3ac9a7db4033584a5b3af32d75bf7473441075a8f500d0312d9602b0ad4c31
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Succe

In [ ]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
print(tf.__version__)
print(keras.__version__)

1.15.0
2.2.4-tf


In [ ]:
train_benign_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/train/benign"
train_malignant_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/train/malignant"
test_benign_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/test/benign"
test_malignant_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/test/malignant"
validation_benign_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/validation/benign"
validation_malignant_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/validation/malignant"

In [ ]:
print('Total number of benign images for training:',len(os.listdir(train_benign_dir)))
print('Total number of malignant images for training:',len(os.listdir(train_malignant_dir)))
print('Total number of benign images for test:',len(os.listdir(test_benign_dir)))
print('Total number of malignant images for test:',len(os.listdir(test_malignant_dir)))
print('Total number of benign images for validation:',len(os.listdir(validation_benign_dir)))
print('Total number of malignant images for validation:',len(os.listdir(validation_malignant_dir)))

Total number of benign images for training: 1140
Total number of malignant images for training: 897
Total number of benign images for test: 360
Total number of malignant images for test: 300
Total number of benign images for validation: 300
Total number of malignant images for validation: 300


In [ ]:
from keras.applications import InceptionV3
myInception=InceptionV3(weights='imagenet',
                 include_top=False,
                 pooling="avg",
                 input_shape=(299,299,3))

Using TensorFlow backend.







86892544/87910968 [============================>.] - ETA: 0sWARNING:tensorflow:From /usr/local/lib/python3.7/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.







In [ ]:
myInception.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 299, 299, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 149, 149, 32)  864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 149, 149, 32)  96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 149, 149, 32)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [ ]:
from keras import models
from keras import layers
model = models.Sequential()
model.add(myInception)
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 2048)              21802784  
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2049      
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
train_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/train"
validation_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/validation"
test_dir="/content/drive/MyDrive/deepLearning/skin_cancer/dataset/test"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary')

Found 2037 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [ ]:
from keras import metrics
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(),
metrics=["acc"])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
from keras.callbacks import ModelCheckpoint
model_path="/content/drive/MyDrive/deepLearning/skin_cancer/saved_models/weights.{epoch:02d}-{val_acc:.2f}.h5"
mc =ModelCheckpoint(filepath=model_path, monitor='val_acc',verbose=1, mode="max", save_best_only=True)
callback_list=[mc]

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=64,
    epochs=10,
    callbacks=callback_list,
    validation_data=validation_generator,
    validation_steps=19)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/10
64/64 [==============================] - 731s - loss: 0.5940 - acc: 0.6992 - val_loss: 8.0597 - val_acc: 0.5000
Epoch 2/10
64/64 [==============================] - 107s - loss: 0.4434 - acc: 0.7922 - val_loss: 7.7638 - val_acc: 0.5183
Epoch 3/10
64/64 [==============================] - 107s - loss: 0.3660 - acc: 0.8239 - val_loss: 4.7165 - val_acc: 0.5883
Epoch 4/10
64/64 [==============================] - 106s - loss: 0.3334 - acc: 0.8344 - val_loss: 7.5314 - val_acc: 0.4967
Epoch 5/10
64/64 [==============================] - 107s - loss: 0.3212 - acc: 0.8410 - val_loss: 3.0944 - val_acc: 0.6233
Epoch 6/10
64/64 [==============================] - 106s - loss: 0.2889 - acc: 0.8705 - val_loss: 5.7969 - val_acc: 0.5083
Epoch 7/10
64/64 [==============================] - 106s - loss: 0.2813 - acc: 0.8718 - val_loss: 3.0108 - val_acc: 0.6133
Epoch 8/10
64/64 [=

In [ ]:
from keras.models import load_model
new_model=load_model("/content/drive/MyDrive/deepLearning/skin_cancer/saved_models/weights.07-0.72.h5")

In [ ]:
history = new_model.fit_generator(
    train_generator,
    steps_per_epoch=64,
    epochs=30,
    callbacks=callback_list,
    validation_data=validation_generator,
    validation_steps=19)

Epoch 1/30
64/64 [==============================] - 141s - loss: 0.2604 - acc: 0.8740 - val_loss: 0.7236 - val_acc: 0.7817
Epoch 2/30
64/64 [==============================] - 106s - loss: 0.2432 - acc: 0.8872 - val_loss: 1.8858 - val_acc: 0.7117
Epoch 3/30
64/64 [==============================] - 106s - loss: 0.2400 - acc: 0.8904 - val_loss: 1.1700 - val_acc: 0.7583
Epoch 4/30
64/64 [==============================] - 106s - loss: 0.2271 - acc: 0.8933 - val_loss: 0.9725 - val_acc: 0.7250
Epoch 5/30
64/64 [==============================] - 107s - loss: 0.2361 - acc: 0.8943 - val_loss: 0.4450 - val_acc: 0.8517
Epoch 6/30
64/64 [==============================] - 106s - loss: 0.2324 - acc: 0.8940 - val_loss: 0.5562 - val_acc: 0.8000
Epoch 7/30
64/64 [==============================] - 106s - loss: 0.2346 - acc: 0.8886 - val_loss: 0.8464 - val_acc: 0.7500
Epoch 8/30
64/64 [==============================] - 106s - loss: 0.2258 - acc: 0.8999 - val_loss: 0.6745 - val_acc: 0.6617
Epoch 9/30
64/64

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary')

Found 660 images belonging to 2 classes.


In [ ]:
from keras.models import load_model
new_model1=load_model("/content/drive/MyDrive/deepLearning/skin_cancer/saved_models/weights.04-0.85.h5")

In [ ]:
test_loss, test_acc = new_model1.evaluate_generator(test_generator, steps=21)
print('test acc:', test_acc)

test acc: 0.8545454552679351
